# Taller #23: Voz al Código: Comandos por Reconocimiento de Voz Local
#### Desarrollado por: David Santiago Cruz Hernández

In [1]:
import speech_recognition as sr
import tkinter as tk
from pythonosc import udp_client
import threading

# pip install -r requirements.txt

### Configuración del cliente OSC

In [2]:
IP_DESTINO = "127.0.0.1"  # IP del servidor (Processing)
PUERTO_OSC = 9100         # Puerto OSC
cliente_osc = udp_client.SimpleUDPClient(IP_DESTINO, PUERTO_OSC)
detener_hilo = threading.Event()

### Lista de comandos permitidos

In [3]:
COMANDOS = ['naranja', 'verde', 'azul', 'mover', 'girar', 'iniciar', 'detener']

### Interfaz gráfica con Tkinter

In [4]:
class AppInteractiva:
    def __init__(self, root):
        self.root = root
        self.root.title("Control por Voz")
        self.root.geometry("500x400")

        self.color_actual = "white"
        self.label = tk.Label(root, text="Esperando comando...", bg=self.color_actual, width=30, height=10, font=("Arial", 20))
        self.label.pack(pady=20)

    def cambiar_color(self, color):
        self.color_actual = color
        self.label.config(bg=color)

    def mostrar_comando(self, comando):
        self.label.config(text=comando.capitalize())

## Función de reconocimiento de voz

In [5]:
def reconocer_voz(app):
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Escuchando...")
        r.adjust_for_ambient_noise(source)
        while not detener_hilo.is_set():
            try:
                audio = r.listen(source, timeout=5, phrase_time_limit=3)
                texto = r.recognize_google(audio, language="es-CO").lower()
                app.root.after(0, app.mostrar_comando, texto)

                if texto in COMANDOS:
                    print(f"Comando reconocido: {texto}")
                    cliente_osc.send_message("/comando", texto)
                    if texto == "naranja":
                        app.root.after(0, app.cambiar_color, "orange")
                    elif texto == "verde":
                        app.root.after(0, app.cambiar_color, "lawn green")
                    elif texto == "azul":
                        app.root.after(0, app.cambiar_color, "turquoise")
                    elif texto == "girar":
                        app.root.after(0, app.mostrar_comando, "Girando...")
                    elif texto == "mover":
                        app.root.after(0, app.mostrar_comando, "Moviendo...")
                    elif texto == "iniciar":
                        app.root.after(0, app.mostrar_comando, "Iniciando...")
                    elif texto == "detener":
                        app.root.after(0, app.mostrar_comando, "Deteniendo...")
                        app.root.after(0, app.cambiar_color, "white")
                else:
                    print(f"Comando no válido: {texto}")
            except sr.UnknownValueError:
                print("No entendido.")
            except sr.RequestError as e:
                print(f"Error en servicio; {e}")
            except Exception as ex:
                print(f"Error: {ex}")

### Iniciar la aplicación Tkinter en un hilo separado

In [6]:
def iniciar_gui():
    root = tk.Tk()
    app = AppInteractiva(root)
    hilo = threading.Thread(target=reconocer_voz, args=(app,))
    hilo.start()
    def on_close():
        detener_hilo.set()
        root.destroy()
    root.protocol("WM_DELETE_WINDOW", on_close)
    root.mainloop()

---
## Ejecutar

In [7]:
if __name__ == "__main__":
    print("Iniciando interfaz y reconocimiento por voz...")
    iniciar_gui()

Iniciando interfaz y reconocimiento por voz...
Escuchando...
Comando no válido: hola
Comando no válido: puedes ver
Comando no válido: lo que estoy diciendo
Comando no válido: así es
Comando no válido: cambiemos los colores
Comando reconocido: naranja
Comando reconocido: azul
Comando reconocido: verde
Comando no válido: agreguemos movimiento
Comando reconocido: girar
Comando reconocido: mover
Comando reconocido: girar
Comando reconocido: detener
Comando reconocido: iniciar
Comando reconocido: girar
Comando reconocido: verde
Comando no válido: qué les parece
Comando no válido: increíble
No entendido.
No entendido.
No entendido.
No entendido.
No entendido.
No entendido.
